# Import Libraries and Files

In [1]:
from Robots.DiscreteDeepRobots import ThreeLinkRobot
from math import pi, log
import os
from csv_generator import generate_csv
from mpl_toolkits.mplot3d import Axes3D
import datetime
import random
import numpy as np
import copy
%matplotlib notebook
# %matplotlib inline
import matplotlib.pyplot as plt

# Q-Learning and Auxiliary Functinos

In [2]:
def Qlearner(num_robots, alpha, gamma, epsilon, theta_lower, theta_upper, a1_lower, a1_upper, 
             a2_lower, a2_upper, a_lower, a_upper, angle_interval, graphs = False, live_graphs = False):
    """
    :param num_robots: number of robots
    :param alpha: learning rate
    :param gamma: discount rate
    :param epsilon: probability of choosing random action while learning
    :param theta_lower: lower limit of theta in state space
    :param theta_upper: upper limit of theta in state space
    :param a1_lower: lower limit of a1 in state space
    :param a1_upper: upper limit of a1 in state space
    :param a2_lower: lower limit of a2 in state space
    :param a2_upper: upper limit of a2 in state space
    :param a_lower: lower limit of action space
    :param a_upper: upper limit of action space
    :param angle_interval: interval of state and action space
    :param graphs: whether to generate graphs
    :param live_graphs: whether to generate dynamics graphs
    :return: a tuple of learned q-values, states, actions
    """

    # initialize state space, action space, and Qvalues
    print('loading state space')
    states = get_state_space(theta_lower, theta_upper,a1_lower,
                             a1_upper, a2_lower, a2_upper, angle_interval)  # state = (theta, a1, a2)
    print(len(states), 'states loaded')
    print('loading action space')
    actions = get_action_space(a_lower, a_upper, angle_interval)  # action = (a1dot, a2dot)
    print(len(actions), 'actions loaded')
    Qvalues = {}
    print('initializing Qvalues')
    for state in states:
        for action in actions:
            Qvalues[(state, action)] = 0
    print(len(Qvalues.keys()), 'q-values loaded')

    # learn q-values
    print('\n\n')
    details = {}
    
    for j in range(num_robots):
        print(j+1, ' ith ', 'robot is learning')
        # state = random.choice(states)
#         robot = ThreeLinkRobot(x=0, 
#                                y=0, 
#                                theta=state[0], 
#                                a1=state[1],
#                                a2=state[2],
#                                link_length=2, 
#                                t_interval=1, 
#                                a_interval=angle_interval)
        robot = ThreeLinkRobot(x=0, 
                               y=0, 
                               theta=0, 
                               a1=pi/16,
                               a2=-pi/16,
                               link_length=2, 
                               t_interval=1, 
                               a_interval=angle_interval)
        i = 0
        
        # plotting
        xs = [robot.x]
        a1s = [robot.a1]
        a2s = [robot.a2]
        timesteps = [i]
        
        for k in range(100):
            i += 1
            old_x = robot.x
            # print('For', j+1, 'th robot', 'In', i, 'th iteration the initial state is: ', state)
            # employ an epsilon-greedy strategy for exploration vs exploitation
            
            best_actions = []
            if random.random() < epsilon:

                # choose a random action
                best_actions = actions
                # randomly select a tie-breaking, valid action
                while True:
                    best_action = random.choice(best_actions)
                    # print('The action randomly chosen is', best_action)
                    temp_robot = copy.deepcopy(robot)
                    temp_robot.move(best_action[0], best_action[1], 1)
                    if (temp_robot.theta, temp_robot.a1, temp_robot.a2) in states:
                        break
            else:

                # find the best actions (the ones with largest q-value)
                maxQ = -float("inf")
                for action in actions:
                    Q = Qvalues[(state, action)]
                    
                    # identify action with higher Q value
                    temp_robot = copy.deepcopy(robot)
                    if Q > maxQ:
                        
                        # check if action is valid
                        temp_robot.move(action[0], action[1], 1)
                        if (temp_robot.theta, temp_robot.a1, temp_robot.a2) in states:
                            best_actions = [action]
                            maxQ = Q
                    elif Q == maxQ:
                        temp_robot.move(action[0], action[1], 1)
                        if (temp_robot.theta, temp_robot.a1, temp_robot.a2) in states:
                            best_actions.append(action)
                    del temp_robot
                
                # randomly select a tie-breaking, valid action
                best_action = random.choice(best_actions)


            # print('In', i, 'th iteration the best action is: ', best_action)
            robot.move(best_action[0],best_action[1],1)

            # transition to new state
            new_state = robot.state
            new_x = robot.x
            # print('In', i, 'th iteration the new state is: ', new_state)
            
            # add values to lists
            xs.append(robot.x)
            a1s.append(robot.a1)
            a2s.append(robot.a2)
            timesteps.append(i)

            # plotting
            if live_graphs:
                make_graphs(xs,a1s,a2s,timesteps,j+1,live=True, Q=True)
            

            # find the maximum Q value for new state
            Q = -float("inf")
            for action in actions:
                Q = max(Q, Qvalues[(new_state, action)])

            # find the reward of this transition
            # reward = 0
            a1, a2, R, a1dot, a2dot = robot.a1, robot.a2, robot.R, robot.a1dot, robot.a2dot
            
            '''
            # penalize according to joint proximity
            if a1 == a2:
                reward += -10*R
            
            else:
                print('In ', i, 'th iteration the penalty for joint angle proximity is: ', log(a1-a2), 'for joint angles: ', a1, a2)
                reward += log(a1-a2)
            '''
            
            # print('In ', i, 'th iteration the reward for x- velocity is: ', v/(a1dot**2 + a2dot**2), 'for velocity, a1dot, a2dot: ', v, a1dot, a2dot)
            # reward += v/(a1dot**2 + a2dot**2)
            reward = 100*(new_x-old_x)

            # TD update
            sample = gamma * Q
            old_Q = Qvalues[(state, best_action)]
            # print('In ', i, 'th iteration the Q value before update is: ', old_Q)
            new_Q = (1 - alpha) * old_Q + alpha * (reward + sample)
            Qvalues[(state, best_action)] = new_Q
            # print('In ', i, 'th iteration the Q value after update is: ', new_Q)
            state = new_state

#             # check for convergence
#             if old_Q == 0:
#                 pass
#             elif abs((new_Q-old_Q)/old_Q) <= 0.05:
#                 print('algorithm converged')
#                 break
        if graphs:
            make_graphs(xs, a1s, a2s, timesteps, j,Q=True)
        details[j] = (xs, a1s, a2s, timesteps)
        
    return Qvalues, states, actions, details

In [3]:
def get_action_space(lower_limit, upper_limit, interval):
    """
    auxiliary function used by Qlearner() to get action space
    :return: a list of action space values in tuple format (a1dot, a2dot)
    """
    upper_limit += (interval/10)  # to ensure the range covers the rightmost value in the loop
    r = np.arange(lower_limit, upper_limit, interval)
    space = [(rnd(i), rnd(j)) for i in r for j in r]

    # remove a1dot = 0, a2dot = 0 from action space
    space.remove((0,0))

    return space


def get_state_space(theta_lower, theta_upper,
                    a1_lower, a1_upper, a2_lower, a2_upper, angle_interval):
    """
    auxiliary function used by Qlearner() to get action space
    :return: a list of state space values in tuple format (a1, a2, theta)
    """
    # to ensure the range covers the rightmost value in the loop
    theta_upper += (angle_interval/10)
    a1_upper += (angle_interval/10)
    a2_upper += (angle_interval/10)

    theta_range = np.arange(theta_lower, theta_upper, angle_interval)
    a1_range = np.arange(a1_lower, a1_upper, angle_interval)
    a2_range = np.arange(a2_lower, a2_upper, angle_interval)
    space = [(rnd(theta), rnd(a1), rnd(a2)) for theta in theta_range for a1 in a1_range for a2 in a2_range]

    return space

In [4]:
def rnd(number):
    return round(number, 8)

In [5]:
def make_graphs(xs, a1s, a2s, timesteps, number, PATH, robot_num, live=False, Q=False):

    # plotting
    fig1 = plt.figure(1)
    if Q:
        fig1.suptitle('Q-learning Monitor for robot ' + str(robot_num))
    else:
        fig1.suptitle('Policy Rollout')
    ax1 = fig1.add_subplot(311)
    ax2 = fig1.add_subplot(312)
    ax3 = fig1.add_subplot(313)
    # fig2 = plt.figure(2)
    # fig2.suptitle('a1 vs a2')
    # ax4 = fig2.add_subplot(111)

    ax1.plot(steps, xs, '.-')
    ax1.set_ylabel('x')
    ax1.set_xlabel('steps')
    ax2.plot(steps, a1s, '.-')
    ax2.set_ylabel('a1')
    ax2.set_xlabel('steps')
    ax3.plot(steps, a2s, '.-')
    ax3.set_ylabel('a2')
    ax3.set_xlabel('steps')
    # ax4.plot(a1s,a2s,'.-')
    # ax4.set_xlabel('a1')
    # ax4.set_ylabel('a2')

    fig1.tight_layout()
    # fig2.tight_layout()
    fig1.tight_layout()
    fig1.savefig(PATH + "/" + str(number) + ' th Policy Rollout.png')
    plt.close(fig1)
 

# Policy and Q-value Testing Functions

In [6]:
def extract_policy(Qvalues, states, actions):
    policy = {}
    for state in states:
        maxQ = -float("inf")
        best_action = None
        for action in actions:
            Q = Qvalues[(state, action)]
            maxQ = max(Q, maxQ)
            if Q == maxQ:
                best_action = action
        policy[state] = best_action
    return policy


def test_policy(robot, policy, timestep=20):
    robot_params = []
    robot_param = [robot.x, robot.y, robot.theta, robot.a1, robot.a2, robot.a1dot, robot.a2dot]
    robot_params.append(robot_param)
    xs = [robot.x]
    a1s = [robot.a1]
    a2s = [robot.a2]
    timesteps = [0]
    plt.title('Policy Rollout')
    for i in range(timestep):
        
        # rollout
        initial_state = robot.state
        print('In', i, 'th iteration the initial state is: ', initial_state)
        old_x = robot.x
        action = policy[initial_state]
        print('In', i, 'th iteration the chosen action is: ', action)
        robot.move(action[0], action[1], 1)
        new_x = robot.x
        print('In ', i, 'th iteration, the robot moved ', new_x - old_x, ' in x direction')
        
        # add values to lists
        xs.append(robot.x)
        a1s.append(robot.a1)
        a2s.append(robot.a2)
        timesteps.append(i+1)
        
        # plotting
        make_graphs(dxs,a1s,a2s,timesteps,0,live=True)
        
        plt.plot(a1s,a2s,'.-')
        plt.xlabel('a1')
        plt.ylabel('a2')
        plt.show()
        
        
    plt.show()
    return dx

# Testing

In [13]:
# learn Q values
Qvalues, states, actions, details = Qlearner(num_robots=300,
        alpha=0.5,
        gamma=0.9,
        epsilon=0.5,
        theta_lower=-pi/8,
        theta_upper=pi/8,
        a1_lower=pi/32,
        a1_upper=pi/8,
        a2_lower=-pi/8,
        a2_upper=-pi/32,
        a_lower=-pi/16,
        a_upper=pi/16,
        angle_interval=pi/16,
        graphs=False,
        live_graphs=False)

loading state space
20 states loaded
loading action space
8 actions loaded
initializing Qvalues
160 q-values loaded



1  ith  robot is learning


KeyboardInterrupt: 

## Graph Analysis of Testing Data

In [8]:
# # graphing Q learning details
# for k in details.keys():
#     xs, a1s, a2s, timesteps = details[k]
#     %matplotlib inline
#     make_graphs(xs, a1s, a2s, timesteps, k+1, Q=True)

In [9]:
# plt.close('all')

In [10]:
# # graph Fourier transform
# for k in details.keys():
#     xs, a1s, a2s, timesteps = details[k]
#     xs_ = np.fft.fft(xs)
#     fig = plt.figure(1)
#     ax1 = fig.add_subplot(311)
#     ax1.plot(timesteps, xs_,'.-')

#     a1s_ = np.fft.fft(a1s)
#     ax2 = fig.add_subplot(312)
#     ax2.plot(timesteps, a1s_,'.-')

#     a2s_ = np.fft.fft(a2s)
#     ax3 = fig.add_subplot(313)
#     ax3.plot(timesteps, a2s_,'.-')
#     # %matplotlib inline
#     plt.show()


## Politcy Rollout

In [ ]:
# extract policy
policy = extract_policy(Qvalues, states, actions)
for state in policy:
    print('state', rnd(state[0]),rnd(state[1]),rnd(state[2]), 'action', rnd(policy[state][0]),rnd(policy[state][1]))

In [ ]:
TIMESTAMP = str(datetime.datetime.now())
TRIAL_NAME = ' DiscreteRL '
TRIAL_NUM = 1
PATH = 'Trials/' + TRIAL_NAME + 'Trial ' + str(TRIAL_NUM) + " " + TIMESTAMP
os.mkdir(PATH)
os.chmod(PATH, 0o0777)

TIMESTEPS = 100
for j in range(1):
    robot = ThreeLinkRobot(x=0, 
                           y=0, 
                           theta=0, 
                           a1=pi/16,
                           a2=-pi/16,
                           link_length=2, 
                           t_interval=1, 
                           a_interval=pi/32)
    xs = [robot.x]
    a1s = [robot.a1]
    a2s = [robot.a2]
    steps = [0]
    # robot.randomize_state(enforce_opposite_angle_signs=True)
    robot_params = []
    robot_param = [robot.x, robot.y, robot.theta, float(robot.a1), float(robot.a2), robot.a1dot, robot.a2dot]
    robot_params.append(robot_param)
    print('Beginning', j+1,  'th Policy Rollout')
    try:
        for i in range(TIMESTEPS):

            # rollout
            state = robot.state
            print('In', i, 'th iteration the initial state is: ', state)
            old_x = robot.x
            action = policy[state]
            print('In', i, 'th iteration the chosen action is: ', action)
            robot.move(action[0], action[1], 1)
            new_x = robot.x
            print('In ', i, 'th iteration, the robot moved ', new_x - old_x, ' in x direction')

            # add values to lists
            xs.append(robot.x)
            a1s.append(robot.a1)
            a2s.append(robot.a2)
            steps.append(i+1)
            robot_param = [robot.x, robot.y, robot.theta, float(robot.a1), float(robot.a2), robot.a1dot, robot.a2dot]
            robot_params.append(robot_param)

    except ZeroDivisionError as e:
        print(str(e), 'occured at ', j+1, 'th policy rollout')

    # plotting
    make_graphs(xs,a1s,a2s,steps,j+1,PATH,0)
    generate_csv(robot_params, PATH + "/" + str(j+1) + " th rollout.csv")